# Image Caption Generation using ViT-GPT2 image captioning model


#### Installing packages

In [ ]:
# installing transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00


In [ ]:
#installing pyngrok
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15979 sha256=41205ad7245b32bbd3ff8dab97e7793553ab8a4706219ee350a7e6c3ca201aed
  Stored in directory: /root/.cache/pip/wheels/89/2d/c2/abe6bcfde6bce368c00ecd73310c11edb672c3eda09a090cfa
Successfully built pyngrok


In [ ]:
#installing streamlit
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 102.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=20b0868b9cb11dbfa5bc36a0f2c547c39bbf9671b42896ecee7637a6522c75c0
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


### Creating app for image captioning

In [ ]:
%%writefile app.py

#importing necessary libraries
import streamlit as st
import tensorflow as tf


st.set_option('deprecation.showfileUploaderEncoding', False)

st.write("""# Instant Image Caption Generator""")

#creating file uploader
file = st.file_uploader("Please upload the image to generate the caption", type = ["jpg", "png"])
 
#creating if-else function to perform the task for captioning. If file is not uploaded the text "Please upload an image file." 
#will be displayed and when the image gets uploaded the captioning task will be performed
if file is None:
    st.text("Please upload an image file.")
else:
    #opens and dispays the uploaded image
    from PIL import Image
    image = Image.open(file)
    st.image(image, use_column_width=True)
    
    #importing necessary libraries to perform image captioning
    from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
    import torch
    import cv2
    from PIL import Image

    #downloading the pretrained model from hugging face.
    ptmodel = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning") #downloading model
    feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")  #downloading feature extractor
    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")  #downloading tokenizer

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ptmodel.to(device)

    #defining the constansts for kwargs
    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
    
    #takes images
    images = []
    i_image = Image.open(file)
    #if the image is not in RGB format this code converts the image to RGB  
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")
    images.append(i_image)

    #generating pixel values (feature extraction)
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    #generating output ids
    output_ids = ptmodel.generate(pixel_values, **gen_kwargs)

    #prediction
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    #dispaying the caption
    string = "The caption for the image is : "+preds.__str__()
    st.success(string)


Writing app.py


In [ ]:
#taking ngrok authentication
!ngrok authtoken 2OtxalV6j9VCDLCzkUX7nKtRvP6_7YykEHpNXzqTticCpq37v

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#running the app
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
#connecting to the website
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url

'http://25d4-34-87-98-56.ngrok-free.app'

In [ ]:
# show the error if there are any
!cat /content/nohup.out